In [5]:
import cv2
import numpy as np

In [112]:
# Count objects for the first image with normal noise
img = cv2.imread("Photos/Rice/1_wIXlvBeAFtNVgJd49VObgQ.png")
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Remove noise
# Seems like, if without opening step, median blur gives a better result compare to gaussian blur
blur = cv2.GaussianBlur(gray, (5, 5), 0)

# After testing the threshold value, I think 119 or 120 would be appropriate bc after thresholding, I apply opening to remove small objects.
_, thresh = cv2.threshold(blur, 119, 255, cv2.THRESH_BINARY)

# Apply opening to remove small objects
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3)) # rectangular structuring element
opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)

# Count object by counting the contour shapes
contours, hierarchy = cv2.findContours(opening, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
objects = str(len(contours))
text = str(objects)
cv2.putText(opening, text, (10, 25), cv2.FONT_HERSHEY_SIMPLEX,
                   0.4, (255, 0, 0), 1, cv2.LINE_AA)
print(objects)

cv2.drawContours(img, contours, -1, (255, 0, 0), 3)
cv2.imshow('Thresh', thresh)
cv2.imshow('Opening', opening)
cv2.imshow('Contours', img)

cv2.waitKey(0)
cv2.destroyAllWindows()

90


In [109]:
# Count objects for the second image with salt pepper noise
img = cv2.imread("Photos/Rice/1_wIXlvBeAFtNVgJd49VObgQ.png_Salt_Pepper_Noise1.png")
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Remove noise using median blur
blur = cv2.medianBlur(gray, 5)

# After testing the threshold value, I think 116 or 117 would be appropriate bc after thresholding, I apply opening to remove small objects.
_, thresh = cv2.threshold(blur, 117, 255, cv2.THRESH_BINARY)

# Apply opening to remove small objects
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5)) # rectangular structuring element
opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)

# Count object by counting the contour shapes
contours, hierarchy = cv2.findContours(opening, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
objects = str(len(contours))
text = str(objects)
cv2.putText(opening, text, (10, 25), cv2.FONT_HERSHEY_SIMPLEX,
                   0.4, (255, 0, 0), 1, cv2.LINE_AA)
print(objects)

cv2.drawContours(img, contours, -1, (255, 0, 0), 3)
cv2.imshow('Thresh', thresh)
cv2.imshow('Opening', opening)
cv2.imshow('Contours', img)

cv2.waitKey(0)
cv2.destroyAllWindows()

89


In [114]:
# Count objects for the third image with sinus noise => cannot use normal filter to blur(remove noise)
img = cv2.imread("Photos/Rice/1_wIXlvBeAFtNVgJd49VObgQ_sinus.png")
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Use FFT (frequency domain)
blur = cv2.GaussianBlur(gray, (5, 5), 0)

# After testing the threshold value, I think 116 or 117 would be appropriate bc after thresholding, I apply opening to remove small objects.
_, thresh = cv2.threshold(blur, 117, 255, cv2.THRESH_BINARY)

# Apply opening to remove small objects
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5)) # rectangular structuring element
opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)

# Count object by counting the contour shapes
contours, hierarchy = cv2.findContours(opening, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
objects = str(len(contours))
text = str(objects)
cv2.putText(opening, text, (10, 25), cv2.FONT_HERSHEY_SIMPLEX,
                   0.4, (255, 0, 0), 1, cv2.LINE_AA)
print(objects)

cv2.drawContours(img, contours, -1, (255, 0, 0), 3)
cv2.imshow('Thresh', thresh)
cv2.imshow('Opening', opening)
cv2.imshow('Contours', img)

cv2.waitKey(0)
cv2.destroyAllWindows()

17
